In [64]:
import json
import pandas as pd

file_path = "prices.jsonl"

# Load the JSON data from the selected file
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Initialize an empty list to store the rows of our DataFrame
df_rows = []

# Process the JSON data to populate the df_rows list
for inputted_location, restaurants in data.items():
    for restaurant_name, details in restaurants.items():
        restaurant_location = details.get('location', '')
        restaurant_rating = details.get('rating', 'Not Rated')  # Default to 'Not Rated' if there's no rating
        restaurant_distance = details.get('distance')
        for item, price in details.get('menu', {}).items():
            df_rows.append({
                'restaurant_name': restaurant_name,
                'menu_item': item,
                'menu_item_price': price,
                'restaurant_location': restaurant_location,
                'inputted_location': inputted_location,
                'restaurant_rating': restaurant_rating,  # Add the rating to each row
                'restaurant_distance': restaurant_distance
            })


# Convert the list of rows into a DataFrame
df = pd.DataFrame(df_rows)


In [65]:
df['restaurant_name'] = df['restaurant_name'].str.replace(r'_[0-9]+$', '', regex=True)

In [66]:
df

,restaurant_name,menu_item,menu_item_price,restaurant_location,inputted_location,restaurant_rating,restaurant_distance
0,McDonald,Medium French Fries,4.89,"4514 TELEGRAPH AVE, Oakland, CA, 94609, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.5,2 mi
1,McDonald,10 pc. Chicken McNuggets®,6.79,"4514 TELEGRAPH AVE, Oakland, CA, 94609, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.5,2 mi
2,McDonald,McChicken®,5.19,"4514 TELEGRAPH AVE, Oakland, CA, 94609, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.5,2 mi
3,McDonald,2 Cheeseburger Meal,10.69,"4514 TELEGRAPH AVE, Oakland, CA, 94609, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.5,2 mi
4,McDonald,Big Mac®,7.39,"4514 TELEGRAPH AVE, Oakland, CA, 94609, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.5,2 mi
...,...,...,...,...,...,...,...
441,McDonald,Spicy McCrispy™,5.69,"1620 STROBRIDGE AVE, CASTRO VALLEY, CA, 94546, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.4,12.7 mi
442,McDonald,Double Bacon Quarter Pounder® with Cheese,9.69,"1620 STROBRIDGE AVE, CASTRO VALLEY, CA, 94546, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.4,12.7 mi
443,McDonald,Double Quarter Pounder® with Cheese Deluxe,9.19,"1620 STROBRIDGE AVE, CASTRO VALLEY, CA, 94546, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.4,12.7 mi
444,McDonald,Deluxe McCrispy™,7.89,"1620 STROBRIDGE AVE, CASTRO VALLEY, CA, 94546, US","1 Frank H. Ogawa Plaza, Oakland, CA 94612",4.4,12.7 mi


In [ ]:
df.to_csv('fast_food_restaurants_menu.csv', index=False) 